In [1]:
import tensorflow as tf
import numpy as np

tf.random.set_seed(777)

In [6]:
xy=np.loadtxt('data-04-zoo.csv',delimiter=',',dtype=np.float32)
x_data=xy[:,0:-1]
y_data=xy[:,-1]

nb_classes=7
Y_one_hot = tf.one_hot(y_data.astype(np.int32),nb_classes)

print(x_data.shape,Y_one_hot.shape)

(101, 16) (101, 7)


In [7]:
W=tf.Variable(tf.random.normal((16,nb_classes)),name='weight')
b=tf.Variable(tf.random.normal((nb_classes,)),name='bias')
variables=[W,b]

def logit_fn(X):
    return tf.matmul(X,W)+b

def hypothesis(X):
    return tf.nn.softmax(logit_fn(X))

def cost_fn(X,Y):
    logits = logit_fn(X)
    cost_i=tf.keras.losses.categorical_crossentropy(y_true=Y,y_pred=logits,from_logits=True)
    
    cost=tf.reduce_mean(cost_i)
    return cost

def grad_fn(X,Y):
    with tf.GradientTape() as tape:
        loss = cost_fn(X,Y)
        grads = tape.gradient(loss,variables)
        return grads
    
def prediction(X,Y):
    pred = tf.argmax(hypothesis(X),1)
    correct_prediction = tf.equal(pred,tf.argmax(Y,1))
    accuracy = tf.reduce_sum(tf.cast(correct_prediction,tf.float32))
    
    return accuracy

In [9]:
def fit(X,Y,epochs=1000,verbose=100):
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)
    
    for i in range(epochs):
        grads=grad_fn(X,Y)
        optimizer.apply_gradients(zip(grads,variables))
        if (i==0) | ((i+1)%100==0):
            acc=prediction(X,Y).numpy()
            loss=cost_fn(X,Y).numpy()
            print('Steps: {} Loss: {}, Acc: {}'.format(i+1,loss,acc))
            
fit(x_data,Y_one_hot)

Steps: 1 Loss: 3.6350290775299072, Acc: 17.0
Steps: 100 Loss: 0.5194158554077148, Acc: 80.0
Steps: 200 Loss: 0.31850090622901917, Acc: 92.0
Steps: 300 Loss: 0.23534876108169556, Acc: 95.0
Steps: 400 Loss: 0.18872138857841492, Acc: 96.0
Steps: 500 Loss: 0.1584603488445282, Acc: 96.0
Steps: 600 Loss: 0.13703757524490356, Acc: 100.0
Steps: 700 Loss: 0.12098980695009232, Acc: 100.0
Steps: 800 Loss: 0.10847964882850647, Acc: 101.0
Steps: 900 Loss: 0.09843039512634277, Acc: 101.0
Steps: 1000 Loss: 0.09016556292772293, Acc: 101.0
